In [ ]:
%pip install awscli

In [ ]:
!aws configure

In [ ]:
!aws s3 cp s3://balldontlie/datasets/intent_classifier/v000/train.json /content/train.json

In [ ]:
%pip install pandas numpy evaluate datasets transformers

In [ ]:
import io
import json
import pandas as pd
import numpy as np
import evaluate
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
with open("/content/train.json", "r") as f:
  data = json.load(f)

df = pd.DataFrame(data[""], columns=["query", "intent"])
df.to_csv("train.csv", index=False)

In [ ]:
dataset = load_dataset('csv', data_files="/content/train.csv")
train_test = dataset['train'].train_test_split(test_size=0.3, seed=27)
test_val = train_test['test'].train_test_split(test_size=0.5, seed=27)

dataset_dict = DatasetDict({
    'train': train_test['train'],
    'validation': test_val['train'],
    'test': test_val['test']
})

intent_labels = ["lookup", "aggregation", "leaderboard", "comparison", "trend", "out_of_scope"]
id2label = {i: label for i, label in enumerate(intent_labels)}
label2id = {label: i for i, label in enumerate(intent_labels)}

In [ ]:
def encode_labels(example):
    example['labels'] = label2id[example['intent']]
    return example

dataset_dict = dataset_dict.map(encode_labels)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
  return tokenizer(
      examples['query'],
      truncation=True,
      padding="max_length",
      max_length=128
  )

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(intent_labels),
    id2label=id2label,
    label2id=label2id
)

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='epoch',
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy'
)

accuracy = evaluate.load('accuracy')

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return accuracy.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

In [ ]:
trainer.save_model('./intent_classifier')